In [17]:
import carla
import time
from intersection_scenario_manager import IntersectionScenario
import numpy as np
import sys
sys.path.append("cartesian_to_frenet/")
from topology_extraction import get_opendrive_tree, get_junction_topology, get_junction_roads_topology
sys.path.append("../../global_route_planner/")
from global_planner import get_client, draw_waypoints, spawn_vehicle

In [18]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

all_waypoints = world.get_map().generate_waypoints(1)
def draw(road_id, lane_id=None, color=[0,255,0]):
    for wp in all_waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            draw_waypoints(world, [wp], 10, color) 
#             print(wp.lane_id)
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            draw_waypoints(world, [wp], 10, color) 
            
tree = get_opendrive_tree(world)
junction_topology = get_junction_topology(tree)
road_topology = get_junction_roads_topology(tree)

Connection to CARLA server established!


In [19]:
SM = IntersectionScenario(client)

Intersection Manager Initialized...


dict_keys([224, 965, 421, 1175, 905, 1162, 139, 1260, 685, 334, 751, 1148, 1050, 53, 599, 1070, 943, 509, 924, 829, 245])

In [20]:
junctionId = 53

In [21]:
ego_vehicle, my_vehicles, incoming_road_lane_id_to_outgoing_lane_id_dict, intersection_topology, ego_key, global_path_wps = SM.reset(num_vehicles=10, warm_start_duration=20, junction_id=junctionId)

generating scenario at junction id:  53
Control handed to system....


In [22]:
incoming_road_lane_id_to_outgoing_lane_id_dict

{(5, 2): [(4, 2, 62, (2,)), (7, 2, 100, (-1, -2))],
 (5, 1): [(8, -1, 104, (-1,)), (4, 1, 62, (1,))],
 (4, -1): [(5, -1, 63, (-1,)), (7, 1, 80, (-1,))],
 (4, -2): [(5, -2, 63, (-2,)), (8, -2, 92, (-1, -2))],
 (7, -2): [(4, 2, 84, (2, 1)), (8, -2, 126, (-2,))],
 (8, 1): [(7, 1, 125, (1,)), (4, 1, 88, (1,))],
 (7, -1): [(5, -1, 96, (1,)), (8, -1, 126, (-1,))],
 (8, 2): [(7, 2, 125, (2,)), (5, -2, 108, (2, 1))]}

In [23]:
intersecting_left, intersecting_right, parallel_same_dir, parallel_opposite_dir = intersection_topology


In [24]:
intersecting_left

[[(7, -2), (126, -2), (8, -2)], [(7, -1), (126, -1), (8, -1)]]

In [25]:
for element in intersecting_right:
    for item in element:
        draw(item[0], item[1])

In [1]:
# if road_id, lane_id belongs to parallel_same_dir please pass in same_dir = True, vice versa
# usage application:
#     for key in list(incoming_road_lane_id_to_outgoing_lane_id_dict.keys()):
#         get_direction_information(key[0], key[1], ...)
        
def get_direction_information(road_id, lane_id, ego_info, in_out_dict, same_dir, intersecting_left, intersecting_right, parallel_same_dir, parallel_opposite_dir):
    draw(road_id, lane_id)
    right_turning_lane = False
    left_turning_lane = False
    right_most_turning_lane = False
    left_to_the_current = False
    right_next_to_the_current = False
    lanes_to_right = intersecting_right if same_dir else intersecting_left
    lanes_to_left = intersecting_left if same_dir else intersecting_right

    for connection in in_out_dict[(road_id, lane_id)]:
        print((connection[0], connection[1]))
        for right_lanes in lanes_to_right:
            if (connection[0], connection[1]) in right_lanes:
                right_turning_lane = True 

        for left_lanes in lanes_to_left:
            if (connection[0], connection[1]) in left_lanes:
                left_turning_lane = True
    
    # determine if the lane is right_most
    lane_group = parallel_same_dir if same_dir else parallel_opposite_dir
    maximum_id_magnitude = 0
    for lane in lane_group:
        maximum_id_magnitude = max(abs(lane[0][1]), maximum_id_magnitude)
    right_most_turning_lane = maximum_id_magnitude == abs(lane_id)
    
    ego_lane = ego_info[1]
    ego_road = [parallel_same_dir[0][0][0], parallel_same_dir[0][1][0], parallel_same_dir[0][2][0]]
    if (road_id in ego_road): 
        if not same_dir: 
            left_to_the_current = True # which could means right_to_the_current = False
            #since this function should only be applied on (road_id, lane_id) in in_out_dict.keys()
            # update all the fields in connecting lanes if needed
            # update_field_in_group(road_id, lane_id, parallel_opposite_dir, left_to_the_current, True)
            if (abs(ego_lane) == 1 and lane_id * ego_lane == 1):
                right_next_to_the_current = True 
                # update_field_in_group(road_id, lane_id, parallel_opposite_dir, right_next_to_the_current, True)

        else:
            if abs(lane_id) < abs(ego_lane):
                left_to_the_current = True #on the same road but with opposite direction
                # update_field_in_group(road_id, lane_id, parallel_opposite_dir, left_to_the_current, True)
            if (abs(ego_lane - lane_id) == 1):
                right_next_to_the_current = True
                # update_field_in_group(road_id, lane_id, parallel_opposite_dir, right_next_to_the_current, True)

            
    
    assert(right_turning_lane != left_turning_lane)
    print("right_turning_lane: ", right_turning_lane)
    print("left_turning_lane: ", left_turning_lane)
    print("left_to_the_ego_for_all_the_current_lanes: ", left_to_the_current)
    print("right_next_to_the_ego_for_all_the_current_lanes: ", right_next_to_the_current)

In [2]:
ego_nearest_waypoint = client.get_world().get_map().get_waypoint(
    ego_vehicle.get_location(), project_to_road=True
)
client.get_world().debug.draw_string(
    ego_nearest_waypoint.transform.location,
    "O",
    draw_shadow=False,
    color=carla.Color(r=255, g=0, b=0),
    life_time=100,
)

NameError: name 'client' is not defined

In [ ]:
ego_vehicle.destroy()
for actor in my_vehicles:
    actor.destroy()